In [2]:
import os
import json
from flask import Flask, render_template
from flask_basicauth import BasicAuth
import matplotlib
matplotlib.use('Agg')  # Use the 'Agg' backend for Matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from Portfilio import Portfolio
from PortfolioReturns import PortfolioReturns
from data_loader import DataLoader

In [3]:

data_loader = DataLoader()
transaction_data, sectors_data, daily_prices = data_loader.run()

In [14]:
daily_prices['BAC']

{'2022-10-24': 35.06999969482422,
 '2022-10-25': 35.38999938964844,
 '2022-10-26': 35.70000076293945,
 '2022-10-27': 35.869998931884766,
 '2022-10-28': 36.18000030517578,
 '2022-10-31': 36.040000915527344,
 '2022-11-01': 36.20000076293945,
 '2022-11-02': 36.09000015258789,
 '2022-11-03': 35.88999938964844,
 '2022-11-04': 36.790000915527344,
 '2022-11-07': 37.0099983215332,
 '2022-11-08': 37.11000061035156,
 '2022-11-09': 36.52000045776367,
 '2022-11-10': 38.130001068115234,
 '2022-11-11': 38.40999984741211,
 '2022-11-14': 37.7599983215332,
 '2022-11-15': 37.70000076293945,
 '2022-11-16': 37.36000061035156,
 '2022-11-17': 37.16999816894531,
 '2022-11-18': 37.189998626708984,
 '2022-11-21': 37.310001373291016,
 '2022-11-22': 37.4900016784668,
 '2022-11-23': 37.61000061035156,
 '2022-11-25': 37.70000076293945,
 '2022-11-28': 36.86000061035156,
 '2022-11-29': 37.0,
 '2022-11-30': 37.849998474121094,
 '2022-12-01': 36.54999923706055,
 '2022-12-02': 36.08000183105469,
 '2022-12-05': 34.47000

In [5]:
portfolio = Portfolio(transaction_data, sectors_data)
portfolio.run()  

In [15]:
portfolio.portfolio_data

,ticker,Amount Paid,Current Amount,Return
0,BAC,2976.4514,3420.220039,14.91%
1,BHB,1338.4800,1397.759956,4.43%
2,BLK,2649.8000,3149.280029,18.85%
3,CVX,2600.8200,2815.559967,8.26%
5,ENB,2676.9100,2598.070011,-2.95%
6,HSY,2666.4300,2389.790024,-10.37%
7,JNJ,2833.3900,2484.720062,-12.31%
8,KMB,2719.2900,2902.199936,6.73%
9,KO,2908.7328,3055.200073,5.04%
10,LOW,2683.8500,2645.520081,-1.43%


In [11]:
portfolio = Portfolio(transaction_data, sectors_data)
portfolio.get_current_holdings()

In [8]:
portfolio.portfolio_data

,ticker,Amount Paid,Current Amount,Return,sector,industry
0,BAC,2976.4514,3420.220039,14.91%,Financials,Diversified Banks
1,BHB,1338.4800,1397.759956,4.43%,Financials,Regional Banks
2,BLK,2649.8000,3149.280029,18.85%,Financials,Asset Management and Custody Banks
3,CVX,2600.8200,2815.559967,8.26%,Energy,Integrated Oil and Gas
4,ENB,2676.9100,2598.070011,-2.95%,Energy,Oil and Gas Storage and Transportation
5,HSY,2666.4300,2389.790024,-10.37%,Consumer Staples,Packaged Foods and Meats
6,JNJ,2833.3900,2484.720062,-12.31%,Healthcare,Pharmaceuticals
7,KMB,2719.2900,2902.199936,6.73%,Consumer Staples,Household Products
8,KO,2908.7328,3055.200073,5.04%,Consumer Staples,Soft Drinks and Non-alcoholic Beverages
9,LOW,2683.8500,2645.520081,-1.43%,Consumer Discretionary,Home Improvement Retail


In [7]:



portfolio_returns = PortfolioReturns(transaction_data.rename(columns={'signed_shares':'shares'}), daily_prices)
total_amounts = portfolio_returns.run()
portfolio_to_plot = portfolio_returns.plot_portfolio(total_amounts, transaction_data['date'].min(), portfolio_returns.today)
portfolio_to_plot = portfolio_to_plot.reset_index()
portfolio_to_plot.columns = ['Date', 'Return']
plt.figure(figsize=(10,6), facecolor='none')
sns.set(style="darkgrid")
ax = sns.lineplot(data=portfolio_to_plot.reset_index(), y='Return', x='Date', color="white")
ax.set_facecolor('none')  # Set the plot background to be transparent
ax.grid(False)  # Remove the grid
plt.xticks(color='white')  # Set x-axis labels to white
plt.yticks(color='white')  # Set y-axis labels to white
plt.xlabel('Date', color='white')  # Set x-axis title to white
plt.ylabel('Return', color='white')  # Set y-axis title to white
plt.title('Portfolio Returns', color='white')  # Set plot title to white
plt.savefig('static/plot.png')
plt.close()


portfolio_table = portfolio.plot_portoflio_tbl()
table_html = portfolio_table.to_html(classes='dataframe', border=2)